In [ ]:
import numpy as np
import torch
from monai.transforms import Compose, EnsureChannelFirstd, ScaleIntensityd, ResizeD, ToTensord,LoadImaged, RandFlipd, RandRotated, RandGaussianNoised
from monai.data import Dataset, DataLoader, NumpyReader
from monai.networks.nets import resnet18
import torch.nn as nn
import torch.optim as optim
import csv
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold

from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    roc_auc_score
)

In [ ]:
# Define data transformations for data augmentation and normalization
train_transforms = Compose([
    LoadImaged(keys=["image"], reader=NumpyReader),
    EnsureChannelFirstd(keys=["image"]),
    ResizeD(keys=["image"], spatial_size=(128, 128, 32)),
    ScaleIntensityd(keys=["image"]),
    RandFlipd(keys=["image"], prob=0.5, spatial_axis=0),
    RandRotated(keys=["image"], range_x=0.1, range_y=0.1, range_z=0.1, prob=0.5),
    RandGaussianNoised(keys=["image"], prob=0.1, mean=0.0, std=0.1),
    ToTensord(keys=["image", "label"]),
])

val_transforms = Compose([
    LoadImaged(keys=["image"], reader=NumpyReader),
    EnsureChannelFirstd(keys=["image"]),
    ResizeD(keys=["image"], spatial_size=(128, 128, 32)),
    ScaleIntensityd(keys=["image"]),
    ToTensord(keys=["image", "label"]),
])

In [ ]:
csv_file = "data_labels.csv"  # your CSV file

all_items = []
with open(csv_file, "r") as f:
    reader = csv.DictReader(f)  # expects columns: image, label
    for row in reader:
        image_path = row["image"]
        label = int(row["label"])
        all_items.append({"image": image_path, "label": label})

# Shuffle the entire dataset
random.shuffle(all_items)

In [ ]:
# Define K-Fold Cross Validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

In [ ]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained 3dResNet-18 model
model = resnet18(
    spatial_dims=3,
    n_input_channels=1,
    num_classes=2
).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# Store final results
all_y_true, all_y_pred, all_y_prob = [], [], []
final_metrics = {}

In [ ]:
# Perform K-Fold Cross Validation
for fold, (train_idx, val_idx) in enumerate(kf.split(all_items)):
    print(f"\n🔹 Starting Fold {fold + 1}/{k}")

    train_subset = [all_items[i] for i in train_idx]
    val_subset = [all_items[i] for i in val_idx]

    train_ds = Dataset(train_subset, transform=train_transforms)
    val_ds = Dataset(val_subset, transform=val_transforms)

    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=8, shuffle=False)

    # Training and validation loop
    num_epochs = 10
    fold_train_loss, fold_val_loss = [], []
    y_true_fold, y_pred_fold, y_prob_fold = [], [], []
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for batch in train_loader:
            images, labels = batch["image"], batch["label"]
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        avg_train_loss = epoch_loss / len(train_loader)
        fold_train_loss.append(avg_train_loss)
        print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss:.4f}")
        
        # Validation after every epoch
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                images, labels = batch["image"], batch["label"]
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                probs = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class "1"
                preds = torch.argmax(outputs, dim=1)
                y_true_fold.extend(labels.cpu().numpy())
                y_pred_fold.extend(preds.cpu().numpy())
                y_prob_fold.extend(probs.cpu().numpy())
        
        avg_val_loss = val_loss / len(val_loader)
        fold_val_loss.append(avg_val_loss)
        print(f"Val Loss: {avg_val_loss:.4f}")
    
    final_metrics["train_loss"].append(np.mean(fold_train_loss))
    final_metrics["val_loss"].append(np.mean(fold_val_loss))
    all_y_true.extend(y_true_fold)
    all_y_pred.extend(y_pred_fold)
    all_y_prob.extend(y_prob_fold)

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(all_y_true, all_y_pred)

# Calculate metrics
acc = accuracy_score(all_y_true, all_y_pred)
prec = precision_score(all_y_true, all_y_pred)
rec = recall_score(all_y_true, all_y_pred)
f1 = f1_score(all_y_true, all_y_pred)

# For ROC-AUC, we need probabilities for the positive class
fpr, tpr, thresholds = roc_curve(all_y_true, all_y_prob)
roc_auc = roc_auc_score(all_y_true, all_y_prob)

print("\n========== Final Evaluation on Validation Set ==========")
print("Confusion Matrix:\n", cm)
print(f"Accuracy:    {acc:.4f}")
print(f"Precision:   {prec:.4f}")
print(f"Recall:      {rec:.4f}")
print(f"F1-Score:    {f1:.4f}")
print(f"ROC-AUC:     {roc_auc:.4f}")

In [ ]:
# Display the results
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Plot the ROC curve
plt.figure(figsize=(5, 4))
plt.plot(fpr, tpr, label=f"ROC curve (AUC={roc_auc:.2f})", color="blue")
plt.plot([0, 1], [0, 1], "r--", label="Random Classifier")
plt.xlim([0, 1])
plt.ylim([0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.show()